# Requête HTTP 

Un requête HTTP est une requête basée sur le protocole TCP, elle fait partie de la couche application de la couche OSI. Elle permet d'accéder aux données mise à disposition sur une adresse IP (ou url résolue par un DNS) et un port. 

Les deux ports les plus utilisés dans le web sont le 80 pour les sites en HTTP et le 443 pour les sites en HTTPS. HTTPS est une variable du protocole HTTP basé sur le protocole TLS.

Il existe de nombreux types de requêtes selon la convention `REST`: 
- GET
- POST
- PUT 
- DELETE
- UPDATE.

Dans notre cas, nous allons utiliser la plupart du temps des GET et potentiellement des POST. 
- Le GET permet comme son nom l'indique de récupérer des informations en fonction de certains paramètres. 
- Le POST nécessite un envoi de données pour récupérer des données. Le body du post est, la plupart du temps, envoyé sous la forme d'un objet JSON.

Ces requêtes encapsulent un certain nombre de paramètres qui permettent soient d'identifier une provenance et un utilisateur ou de réaliser différentes actions.

In [ ]:
import requests

In [ ]:
url = "https://www.esiee.fr/"
response = requests.get(url)
response.status_code

Il existe deux méthodes pour récupérer le contenu de la page :

- `response.text` qui permet de retourner le texte sous la forme d'une chaine de charactères.
- `response.content` qui permet de récupérer le contenu de la page sous la forme de bytes

In [ ]:
type(response.content)

In [ ]:
type(response.text)

Pour récupérer les 1000 premiers charactères de la page :

In [ ]:
response.text[0:1000]

Pour récupérer les headers HTTP de la réponse :

In [ ]:
response.headers

On peut modifier les paramètres de la requête et/ou ses headers. On peut par exemple ajouter un UserAgent (identifiant de l'initiateur de la requête) et un timeout de 10 secondes :

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers, timeout = 10)
response.content[0:1000]

## Exercice

## Exercice 1

- Créer une classe Python permettant de faire des requêtes HTTP.
- Cette classe doit utiliser toujours le même UserAgent.
- Le TimeOut sera spécifié à chaque appelle avec une valeur par défaut.
- Un mécanisme de retry sera mis en place de façon recursive.

## Exercice 2

- Faire une fonction permettant de supprimer tous les espaces supperflus d'une string
- Faire une fonction qui prend une string html et renvois une string intelligible (enlever les caractères spéciaux,
- Récupérer le domaine en fonction d'un url

In [ ]:
# Exercice 1 
# - Créer une classe Python permettant de faire des requêtes HTTP.
# - Cette classe doit utiliser toujours le même UserAgent.
# - Le TimeOut sera spécifié à chaque appelle avec une valeur par défaut.
# - Un mécanisme de retry sera mis en place de façon recursive.

class HttpRequester:
    def __init__(self, user_agent: str):
        self.headers = {'User-Agent': user_agent}

    def get(self, url: str, timeout: int = 10, retries: int = 3):
        try:
            response = requests.get(url, headers=self.headers, timeout=timeout)
            response.raise_for_status()  # Raise an error for bad responses
            return response
        except requests.RequestException as e:
            if retries > 0:
                return self.get(url, timeout, retries - 1)
            else:
                raise e
            
# Example usage:
http_requester = HttpRequester('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36')
response = http_requester.get("https://www.esiee.fr/")
print(response.status_code)


In [ ]:
# Exercice 2
# - Faire une fonction permettant de supprimer tous les espaces supperflus d'une string
# - Faire une fonction qui prend une string html et renvois une string intelligible (enlever les caractères spéciaux,
# - Récupérer le domaine en fonction d'un url

import re
from urllib.parse import urlparse

def remove_extra_spaces(text: str) -> str:
    return ' '.join(text.split())

def clean_html_string(html: str) -> str:
    text = re.sub(r'<[^>]+>', '', html)  # Remove HTML tags
    text = re.sub(r'&[a-z]+;', ' ', text)  # Remove HTML entities
    return remove_extra_spaces(text)

def get_domain_from_url(url: str) -> str:
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Example usage:
html_string = "<html>   <body> Hello &nbsp; World! </body> </html>"
cleaned_string = clean_html_string(html_string)
print(cleaned_string)  # Output: "Hello World!"

# Exploitation du HTML  

Ici, il faut récupérer le code HTML d'un site web à partir d'une requête. Lorsque vous avez récupéré le texte d'un site il faut le parser. Pour cela, on utilise BeautifulSoup qui permet de transformer la structure HTML en objet Python. Cela permet de récupérer efficacement les données qui nous intéresse.

Pour les webmasters, le blocage le plus souvent mis en place et un blocage sur le User-Agent. Le User-Agent est un paramètre intégré dans la requête HTTP réalisé par le Navigateur pour envoyer au front des informations basiques :

- la version du Navigateur,
- la version de l'OS
- Le type de gestionnaire graphique (Gecko)
- le type de device utilisé

Exemple de User Agent :  

`Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0`

Commençons à utiliser `BeautifulSoup`, il est normalement déjà installé, le cas échéant executez les lignes suivantes : 

In [19]:
import requests
from bs4 import BeautifulSoup

Pour transformer une requête (requests) en objet BeautifulSoup :

In [20]:
response = requests.get(url)
soup = BeautifulSoup(response.text)

Pour trouver tous les liens d'une page, on récupère la balise `a` qui permet de gérer les liens en HTML :

In [21]:
soup.find_all("a")[0:10]

[<a href="/#content">Aller au contenu</a>,
 <a href="/#menu">Aller au menu</a>,
 <a href="/plan-du-site/">Plan du site</a>,
 <a href="/actualites/journees-portes-ouvertes-2025-2026" target="_blank" title="Ouvre une nouvelle fenêtre">Journée portes ouvertes le 6 décembre de 13h à 18h. Inscrivez-vous dès maintenant !</a>,
 <a href="/"><img alt="ESIEE PARIS" class="a42-ac-replace-img" src="/typo3conf/ext/esiee_sitepackage/Resources/Public/imgs/svg/logo-esiee.svg"/></a>,
 <a href="/brochures-1">Brochures</a>,
 <a href="/informations/etudiantes-et-etudiants">Espace élèves</a>,
 <a href="/" hreflang="fr-FR" title="Français">
 <span>Fr</span>
 </a>,
 <a href="/en/" hreflang="en-US" title="English">
 <span>En</span>
 </a>,
 <a href="/candidater-1">Candidater</a>]

On peut aussi préciser la classe HTML qu'on veut récupérer :

```python
soup.find_all(class_="<CLASS_NAME>")[0:10]
```

Ici par exemple: 

In [22]:
soup.find_all(class_="toggler")[0:5]

[<button aria-controls="searchbox-header-form" aria-expanded="false" class="toggler">
 <i class="fa-solid fa-magnifying-glass"></i>
 <i class="fa-solid fa-xmark"></i>
 <span class="sr-only">
 <span class="display">Afficher</span><span class="hide">Masquer</span> la recherche
 		</span>
 </button>,
 <button aria-controls="submenu-40" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>L'école</button>,
 <button aria-controls="submenu-563" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>Gouvernance et conseils</button>,
 <button aria-controls="submenu-65" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>Départements d'enseignements et de recherche</button>,
 <button aria-controls="su

Pour récupérer le text sans les balises HTML :

In [23]:
soup.text[0:1000]

"\n\n\n\nESIEE Paris, l'école de l'innovation technologique | ESIEE Paris\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAller au contenu\nAller au menu\nPlan du site\n\n\n\n\n\n\n\nJournée portes ouvertes le 6 décembre de 13h à 18h. Inscrivez-vous dès maintenant !\n\n\n\n\n\nMasquer l'alerte\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBrochuresEspace élèves\n\n\n\nFr\n\n\n\n\nEn\n\n\n\n\n\n\n\n\n\nAfficherMasquer la recherche\r\n\t\t\n\n\n\nSaisissez votre recherche\xa0:\n\nLancer la recherche\n\n\n\nCandidater\n\nAfficherMasquer le menu\n\n\n\n\n\nRetour au menu principalAfficherMasquer le sous menu\xa0: L'écolePourquoi choisir ESIEE Paris ?AfficherMasquer le sous menu\xa0: Gouvernance et conseilsGouvernance et conseilsConseil scientifiqueAfficherMasquer le sous menu\xa0: Départements d'enseignements et de rechercheInformatique et télécommunicationsIngénierie des systèmes cyberphysiquesIngénierie industrielleSanté, énergie et environnement durableManagement, sciences humaines et languesCorps pr

## Exercice
### Exercice 3

Améliorer la classe développé précédemment.

- Ajouter une méthode pour récupérer l'objet soup d'un url
- Récupérer une liste de User Agent et effectuer une rotation aléatoire sur celui à utiliser
- Utiliser cette classe pour parser une page HTML et récupérer : le titre, tous les H1 (si ils existent), les liens vers les images, les liens sortants vers d'autres sites, et le texte principal.

In [24]:
# Exercice 3

# Améliorer la classe développé précédemment.

# - Ajouter une méthode pour récupérer l'objet soup d'un url
# - Récupérer une liste de User Agent et effectuer une rotation aléatoire sur celui à utiliser
# - Utiliser cette classe pour parser une page HTML et récupérer : le titre, tous les H1 (si ils existent), les liens vers les images, les liens sortants vers d'autres sites, et le texte principal

import random
from bs4 import BeautifulSoup

class EnhancedHttpRequester(HttpRequester):
    USER_AGENTS = [
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
        # Add more user agents as needed
    ]

    def __init__(self):
        user_agent = random.choice(self.USER_AGENTS)
        super().__init__(user_agent)

    def get_soup(self, url: str, timeout: int = 10, retries: int = 3) -> BeautifulSoup:
        response = self.get(url, timeout, retries)
        return BeautifulSoup(response.text, 'html.parser')
    
    def parse_page(self, url: str):
        soup = self.get_soup(url)
        title = soup.title.string if soup.title else 'No title'
        h1_tags = [h1.get_text() for h1 in soup.find_all('h1')]
        image_links = [img['src'] for img in soup.find_all('img') if 'src' in img.attrs]
        external_links = [a['href'] for a in soup.find_all('a', href=True) if urlparse(a['href']).netloc and urlparse(a['href']).netloc != urlparse(url).netloc]
        main_text = clean_html_string(soup.get_text())
        
        return {
            'title': title,
            'h1_tags': h1_tags,
            'image_links': image_links,
            'external_links': external_links,
            'main_text': main_text
        }
    
# Example usage:
enhanced_requester = EnhancedHttpRequester()
parsed_data = enhanced_requester.parse_page("https://www.esiee.fr/")
print(parsed_data['title'])

ESIEE Paris, l'école de l'innovation technologique | ESIEE Paris


# Exploitation des appels d'API



Losque le front du site récupère des données sur une API gérée par le back, un appel d'API est réalisé. Cet appel est recensé dans les appels réseaux. Il est alors possible de re-jouer cet appel pour récupérer à nouveau les données. Il est très facile de récupérer ces appels dans l'onglet Network de la console développeur de Chrome ou FireFox. La console vous permet de copier le code CURL de la requête et vous pouvez ensuite la transformer en code Python depuis le site https://curl.trillworks.com/.

Souvent les APIs sont bloquées avec certains paramètres. L'API vérifie que dans les headers de la requête HTTP ces paramètres sont présents :
* un token généré à la volée avec des protocoles OAuth2 (ou moins développés).
* un referer provenant du site web (la source de la requête), très facile à falsifier.



## Exercice 
### Exercice 4

- Utiliser les informations développées plus haut pour récupérer les premiers résultats d'une recherche d'une requête
sur Google. 

Tips : 

- Ouvrir les outils de développements de Chrome ou Firefox
- Onglet Network
- Fouiller dans les requêtes pour voir à quoi ressemble un appel API Google
- Utilisez beautiful soup pour convertir le contenu de la request en objet et accéder aux balises

In [ ]:
#Exercice 4
#- Utiliser les informations développées plus haut pour récupérer les premiers résultats d'une recherche d'une requête sur Google. 

# Tips : 

# - Ouvrir les outils de développements de Chrome ou Firefox
# - Onglet Network
# - Fouiller dans les requêtes pour voir à quoi ressemble un appel API Google
# - Utilisez beautiful soup pour convertir le contenu de la request en objet et accéder aux balises


import urllib.parse
def google_search(query: str, num_results: int = 10):
    search_url = "https://www.google.com/search"
    params = {
        'q': query,
        'num': num_results
    }
    full_url = f"{search_url}?{urllib.parse.urlencode(params)}"
    
    enhanced_requester = EnhancedHttpRequester()
    soup = enhanced_requester.get_soup(full_url)
    
    results = []
    for g in soup.find_all('div', class_='g'):
        title = g.find('h3')
        link = g.find('a', href=True)
        if title and link:
            results.append({
                'title': title.get_text(),
                'link': link['href']
            })
    
    return results
# Example usage:
search_results = google_search("ESIEE Paris", num_results=5)
for result in search_results:
    print(result['title'], result['link'])

# Exercice Final  

Exercice Final Utilisez tout ce que vous avez appris pour récupérer des articles de News avec une catégorie. Il est souvent intéressant de partir des flux RSS pour commencer :

Les données doivent comprendre :

- Le texte important propre
- L'url
- Le domaine
- la catégorie
- Le titre de l'article
- Le titre de la page
- (Facultatif): les imames

Tips :
- Taper le nom de votre média favoris + RSS (par exemple: https://www.lemonde.fr/rss/)
- Aller dans le DOM de la page
- Trouver les catégories et les liens vers les articles




In [27]:
from urllib.parse import urljoin

import xml.etree.ElementTree as ET

def find_rss_feed(site_url: str, requester: EnhancedHttpRequester) -> str | None:
    """Try to discover an RSS/Atom feed for the given site_url."""
    soup = requester.get_soup(site_url)
    # look for link rel alternate RSS/Atom
    for link in soup.find_all("link", rel=lambda r: r and "alternate" in r):
        t = (link.get("type") or "").lower()
        if "rss" in t or "atom" in t or "xml" in t:
            href = link.get("href")
            if href:
                return urljoin(site_url, href)
    # try common feed paths
    common_paths = ["/rss", "/rss.xml", "/feed", "/feeds", "/atom.xml"]
    for p in common_paths:
        candidate = urljoin(site_url, p)
        try:
            resp = requester.get(candidate, timeout=8, retries=1)
            if resp and resp.status_code == 200 and ("xml" in (resp.headers.get("Content-Type") or "") or resp.text.strip().startswith("<")):
                return candidate
        except Exception:
            continue
    return None

def parse_feed_xml(feed_xml: str) -> list[ET.Element]:
    """Return list of item/entry elements from feed xml string."""
    root = ET.fromstring(feed_xml.encode("utf-8"))
    items = root.findall(".//item")
    if not items:
        # try atom entries
        items = root.findall(".//{http://www.w3.org/2005/Atom}entry")
        if not items:
            # fallback: any <entry> without namespace
            items = root.findall(".//entry")
    return items

def extract_text_from_element(el: ET.Element) -> str:
    """Serialize element to HTML and strip tags using BeautifulSoup for readable text."""
    raw = ET.tostring(el, encoding="utf-8", method="html")
    text = BeautifulSoup(raw, "html.parser").get_text(separator=" ")
    return clean_html_string(text)

def get_first_text(e: ET.Element, tag_names: list[str]) -> str | None:
    for tag in tag_names:
        # try with and without common namespaces
        node = e.find(tag)
        if node is not None and (node.text and node.text.strip()):
            return node.text.strip()
        # try namespaced variants
        for child in e:
            if child.tag.lower().endswith(tag.lower()):
                if child.text and child.text.strip():
                    return child.text.strip()
    return None

def extract_link_from_item(item: ET.Element) -> str | None:
    # 1) <link>text</link>
    link = get_first_text(item, ["link"])
    if link and link.startswith("http"):
        return link
    # 2) <link href="..."> (Atom)
    for child in item.findall("link"):
        href = child.get("href")
        if href:
            return href
    # 3) search for enclosure/url
    enc = item.find("enclosure")
    if enc is not None and enc.get("url"):
        return enc.get("url")
    return None

def scrape_news_from_site(site_url: str, max_articles: int = 10) -> list[dict]:
    requester = EnhancedHttpRequester()
    feed_url = find_rss_feed(site_url, requester)
    if not feed_url:
        raise RuntimeError(f"No RSS/Atom feed found for {site_url}")
    resp = requester.get(feed_url)
    items = parse_feed_xml(resp.text)
    results = []
    for item in items[:max_articles]:
        try:
            article_title = get_first_text(item, ["title"]) or ""
            article_link = extract_link_from_item(item)
            category = get_first_text(item, ["category"]) or ""
            # try to extract description/content from feed item
            description = get_first_text(item, ["description"]) or ""
            # sometimes content:encoded is used
            if not description:
                # search for any child that endswith 'encoded' or 'content'
                for child in item:
                    if child.tag.lower().endswith("encoded") or child.tag.lower().endswith("content"):
                        if child.text:
                            description = child.text.strip()
                            break
            main_text = clean_html_string(description) if description else ""
            page_title = ""
            images = []
            if article_link:
                # fetch article to enrich data
                try:
                    page_soup = requester.get_soup(article_link, timeout=10, retries=2)
                    page_title = page_soup.title.string.strip() if page_soup.title else ""
                    images = [urljoin(article_link, img.get("src")) for img in page_soup.find_all("img") if img.get("src")]
                    if not main_text:
                        # fallback to page text
                        main_text = clean_html_string(page_soup.get_text())
                except Exception:
                    pass
            domain = get_domain_from_url(article_link or site_url)
            results.append({
                "article_title": article_title,
                "url": article_link or "",
                "domain": domain,
                "category": category,
                "page_title": page_title,
                "text": remove_extra_spaces(main_text),
                "images": images
            })
        except Exception:
            continue
    return results

# Example usage with provided `url` (and available classes/functions in the notebook):
news = scrape_news_from_site(url, max_articles=8)
for i, item in enumerate(news, 1):
    print(i, item["article_title"], "-", item["url"])

1 ESIEE Paris : deux filières renouvellent leur label SecNumedu par l’ANSSI - https://www.esiee.fr/actualites/esiee-paris-deux-filieres-renouvellent-leur-label-secnumedu-par-lanssi
2 Des élèves ESIEE Paris participent au Concours jeunes talents Orange 2025 - https://www.esiee.fr/actualites/des-eleves-esiee-paris-participent-au-concours-jeunes-talents-orange-2025
3 Préparez votre rentrée ! - https://www.esiee.fr/actualites/preparez-votre-rentree-2025
4 Bienvenue aux admis CPGE 2025 à ESIEE Paris - https://www.esiee.fr/actualites/admissions-cpge-2025
5 L'université Gustave Eiffel parmi les 3% des meilleures universités au monde selon CWUR - https://www.esiee.fr/actualites/luniversite-gustave-eiffel-parmi-les-meilleures-universites-au-monde-selon-cwur
6 Résultats Parcoursup : félicitations aux nouveaux admis à ESIEE Paris - https://www.esiee.fr/actualites/resultats-parcoursup-felicitations-aux-nouveaux-admis-esiee-paris
7 Taxe d'apprentissage 2025 : encouragez l'innovation en choisissant 